In [181]:
import os
from datetime import datetime
import random

In [182]:
class Partition:
    
    def __init__(self, name, identifier, size, usedSpace, freeSpace, blockSize):
        self.name = name
        self.identifier = identifier
        self.size = size
        self.usedSpace = usedSpace
        self.freeSpace = freeSpace
        self.blockSize = blockSize
        
    def toString(self):
        print("- Name :", self.name)
        print("- Identifier :", self.identifier)
        print("- Size :", self.size)
        print("- Used space :", self.usedSpace)
        print("- Free space :", self.freeSpace)
        print("- Block size :", self.blockSize)

In [183]:
class Directory:
    
    def __init__(self, name, identifier, location, size, protection,
                 creation_time, modification_time, access_time):
        self.name = name
        self.identifier = identifier
        self.location = location
        self.size = size
        self.protection = protection
        self.creation_time = creation_time
        self.modification_time = modification_time
        self.access_time = access_time
        
    def toString(self):
        print("- Name :", self.name)
        print("- Identifier :", self.identifier)
        print("- Location :", self.location)
        print("- Size :", self.size)
        print("- Protection :", self.protection)
        print("- Creation time :", self.creation_time)
        print("- Modification time :", self.modification_time)
        print("- Access time :", self.access_time)

In [184]:
class File:
    
    def __init__(self, name, identifier, location, file_type, size, blocks, protection, 
                 creation_time, modification_time, access_time, content):
        self.name = name
        self.identifier = identifier
        self.location = location
        self.file_type = file_type
        self.size = size
        self.blocks = blocks
        self.protection = protection
        self.creation_time = creation_time
        self.modification_time = modification_time
        self.access_time = access_time
        self.content = content
        
    def toString(self):
        print("- Identifier :", self.identifier)
        print("- Type :", self.file_type)
        print("- Size :", self.size)
        print("- Blocks :", self.blocks)
        print("- Protection :", self.protection)
        print("- Creation time :", self.creation_time)
        print("- Modification time :", self.modification_time)
        print("- Access time :", self.access_time)
        print("- Content :", self.content)

In [201]:
class FileSystem:
    
    def __init__(self):
        self.partition = Partition("C", "0", 5000, 0, 5000, 4)
        self.root = {"name": "C", "type": "partition", "partition": self.partition, "children": []}
        self.blockSize = self.root["partition"].blockSize
        
    # files
    def createFile(self, file):
        directoryNode = self.tracePath(file.location)
        # now we're in the directory where we can create the file
        file.size = len(file.content)
        file.blocks = self.calculateNumberOfBlockForEachFile(file.size)
        directoryNode["children"].append({"name": file.name, "type": "file", "file": file})
        # update the size of each folder in path
        self.updateEachDirectorySizeInPath(file.location, file.size)
        print("- File", file.name, "was created at", file.location)
        print("-----------------------")
        
    def fileExists(self, filename, fileLocation):
        filePath = fileLocation.split(os.path.sep)
        # trace the directories
        directoryNode = self.tracePath(fileLocation)
        # we've reached the directory
        fileNode = self.selectFileFromDirectoryByName(filename, directoryNode)
            
        return fileNode != {}

    def readFile(self, filename, fileLocation):
        fileNode = self.accessFile(filename, fileLocation)
        if(fileNode):
            if(fileNode["file"].protection == "W"):
                print("- File", filename, "at directory", fileLocation, "- Write-only -")
                print("-----------------------")
                return
            
            print("Content of file", filename, "at directory", fileLocation, ":")
            print(fileNode["file"].content)
        else:
            print("- File", filename, "at directory", fileLocation, "- not found -")
            
        print("-----------------------")

    def writeToFile(self, filename, fileLocation, newContent):
        fileNode = self.accessFile(filename, fileLocation)
        if(fileNode):
            if(fileNode["file"].protection == "R"):
                print("- File", filename, "at directory", fileLocation, "- Read-only -")
                print("-----------------------")
                return
        
            self.updateEachDirectorySizeInPath(fileLocation, (-1*fileNode["file"].size))
            fileNode["file"].content = newContent;
            fileNode["file"].size = len(newContent);
            fileNode["file"].blocks = self.calculateNumberOfBlockForEachFile(fileNode["file"].size)
            fileNode["file"].modification_time = datetime.now()
            self.updateEachDirectorySizeInPath(fileLocation, fileNode["file"].size)
            print("- Content of file", filename, "at directory", fileLocation, "has been modified")
        else:
            print("- File", filename, "at directory", fileLocation, "- not found -")
            
        print("-----------------------")
    
    def copyFile(self, filename, fileLocation, destinationLocation):
        fileNode = self.accessFile(filename, fileLocation)
        if(fileNode):
            destinationDirectoryNode = self.tracePath(destinationLocation)
            sourceDirectoryNode = self.tracePath(fileLocation)
            # create a copied file instance
            sourceNode = self.selectFileFromDirectoryByName(filename, sourceDirectoryNode)
            sourceFile = sourceNode["file"]
            copyFileIdentifier = random.randint(0, 400)
            copyFile = File(
                name=filename,
                identifier=copyFileIdentifier,
                location=destinationLocation,
                file_type=sourceFile.file_type,
                size=sourceFile.size,
                blocks=sourceFile.blocks,
                protection=sourceFile.protection,
                creation_time=datetime.now(),
                modification_time=datetime.now(),
                access_time=datetime.now(),
                content=sourceFile.content
            )
            destinationDirectoryNode["children"].append({"name": filename, "type": "file", "file": copyFile})
            self.updateEachDirectorySizeInPath(destinationLocation, copyFile.size)
            print("- File", filename, "at", fileLocation, "was copied to", destinationLocation)
        else:
            print("- File", filename, "at directory", fileLocation, "- not found -")
            
        print("-----------------------")
        
    def moveFile(self, filename, fileLocation, destinationLocation):
        fileNode = self.accessFile(filename, fileLocation)
        if(fileNode):
            # get the file from the source location then delete it
            sourceDirectoryNode = self.tracePath(fileLocation)
            sourceNode = self.selectFileFromDirectoryByName(filename, sourceDirectoryNode)
            sourceFile = sourceNode["file"]
            sourceDirectoryNode = self.deleteFileFromDirectory(filename, sourceDirectoryNode)
            # move the file to the destination location
            destinationDirectoryNode = self.tracePath(destinationLocation)
            destinationDirectoryNode["children"].append({"name": filename, "type": "file", "file": sourceFile})
            self.updateEachDirectorySizeInPath(fileLocation, (-1*sourceFile.size))
            self.updateEachDirectorySizeInPath(destinationLocation, sourceFile.size)
            sourceFile.location = destinationLocation
            print("- File", filename, "at", fileLocation, "was moved to", destinationLocation)
        else:
            print("- File", filename, "at directory", fileLocation, "- not found -")
            
        print("-----------------------")
        
    def renameFile(self, filename, fileLocation, newName):    
        fileNode = self.accessFile(filename, fileLocation)
        if(fileNode):
            if(fileNode["file"].protection == "R"):
                print("- File", filename, "at directory", fileLocation, "- Read-only -")
                print("-----------------------")
                return
            
            # get the file from the source location
            sourceDirectoryNode = self.tracePath(fileLocation)
            sourceNode = self.selectFileFromDirectoryByName(filename, sourceDirectoryNode)
            sourceNode["name"] = newName
            sourceNode["file"].name = newName
            sourceNode["file"].modification_time = datetime.now()
            print("- File", filename, "at", fileLocation, "was renamed to", newName)
        else:
            print("- File", filename, "at directory", fileLocation, "- not found -")
            
        print("-----------------------")
            
    def deleteFile(self, filename, fileLocation):
        fileNode = self.accessFile(filename, fileLocation)
        if(fileNode):
            # get the file from the source location
            sourceDirectoryNode = self.tracePath(fileLocation)
            sourceNode = self.deleteFileFromDirectory(filename, sourceDirectoryNode)
            self.updateEachDirectorySizeInPath(fileLocation, (-1*sourceFile.size))
            
            print("- File", filename, "at", fileLocation, "was deleted")
        else:
            print("- File", filename, "at directory", fileLocation, "- not found -")
            
        print("-----------------------")
        
    def getFileProperties(self, filename, fileLocation):
        fileNode = self.accessFile(filename, fileLocation)
        if(fileNode):
            # get the file from the source location
            sourceDirectoryNode = self.tracePath(fileLocation)
            fileNode = self.selectFileFromDirectoryByName(filename, sourceDirectoryNode)
            print("File", filename, "at", fileLocation, "Properties:")
            file = fileNode["file"]
            file.toString()
        else:
            print("- File", filename, "at directory", fileLocation, "- not found -")
        print("-----------------------")
        
    def changeFilePermissions(self, filename, fileLocation, newProtection):
        fileNode = self.accessFile(filename, fileLocation)
        if(fileNode):
            # get the file from the source location
            sourceDirectoryNode = self.tracePath(fileLocation)
            fileNode = self.selectFileFromDirectoryByName(filename, sourceDirectoryNode)
            fileNode["file"].protection = newProtection
            fileNode["file"].modification_time = datetime.now()
            print("- File", filename, "at", fileLocation, "permission changed to", newProtection)
        else:
            print("- File", filename, "at directory", fileLocation, "- not found -")
        print("-----------------------")
        
    # directory
    def deleteDirectory(self, dirLocation):
        path = dirLocation.split(os.path.sep)[1:]
        currentNode = self.root
        for directory in path[:-1]:
            # move the currentNode pointer to the current directory
            currentNode = [child for child in currentNode["children"] if child["name"] == directory][0]
            
        for child in currentNode["children"]:
            if(child["name"] == path[-1] and child["type"] == "directory"):
                self.updateEachDirectorySizeInPath(dirLocation, (-1*child["directory"].size))
                currentNode["children"].remove(child)
                print("- Folder", dirLocation, "was deleted") 
                print("-----------------------")
                return
         
        print("- Folder", dirLocation, "- Not found -")  
        print("-----------------------")
        
    def changeDirectoryPermissions(self, dirLocation, newProtection):
        path = dirLocation.split(os.path.sep)[1:]
        dirNode = self.traceDirectory(path)
        for child in dirNode["children"]:
            if(child["name"] == path[-1] and child["type"] == "directory"):
                dirNode["directory"].protection = newProtection
                dirNode["directory"].modification_time = datetime.now()
                print("- Directory", dirLocation, "permission changed to", newProtection)
                print("-----------------------")
                return
         
        print("- Folder", dirLocation, "- Not found -")  
        print("-----------------------")
    
    def getDirectoryProperties(self, dirLocation):
        path = dirLocation.split(os.path.sep)[1:]
        dirNode = self.traceDirectory(path)

        if(dirNode):
            # it's the portition node
            if(dirNode == self.root):
                print("Partition", dirLocation, "Properties:")
                dirNode["partition"].toString()
                print("-----------------------")
                return

            # it's a directory
            if(len(path) > 1):
                print("Directory", dirLocation, "Properties:")
                for child in dirNode["children"]:
                    if(child["name"] == path[-1] and child["type"] == "directory"):
                        child["directory"].toString()
                        return
            elif(len(path) == 1):
                dirNode["directory"].toString()
                return
         
        print("- Folder", dirLocation, "- Not found -")  
        print("-----------------------")
        
    def searchInDirectories(self, search):
        results = []
        self.findInFileTree(results, search, self.root)
        if(len(results) != 0):
            print("Found", search, "in :")
            for result in results:
                print("-", result)
            print("-----------------------")
            return
        
        print("No results for", search)
        print("-----------------------")
        
    def createDirectory(self, dirName, dirLocation):
        path = dirLocation.split(os.path.sep)[1:]
        dirNode = self.traceDirectory(path)        
        if(dirNode):
            newDirectory = self.createNewDirectoryInstance(dirLocation, dirName)
            dirNode["children"].append({"name": dirName, "type": "directory", "directory": newDirectory, "children": []})
            print("- Directory", dirName, "at", dirLocation, "was created")
        else:
            print("- Directory", dirLocation, "- Not found -")
        
            
    def getDirectoryContent(self, dirLocation):
        path = dirLocation.split(os.path.sep)[1:]
        dirNode = self.traceDirectory(path)
        
        if(dirNode):
            print("Directory", dirLocation, "contains :")
            for child in dirNode["children"]:
                name = child[child["type"]].name
                size = child[child["type"]].size
                print("-", name, "(", size ,"bytes )")
            print("-----------------------")
        
        else:
            print("- Folder", dirLocation, "- Not found -")  
            print("-----------------------")
            
    def displayFileTree(self):
        
        print("---------- File Tree ----------")
        self.generateFileTree(self.root, 0)
        print("-------------------------------")
    
    # helpers
    def accessFile(self, filename, fileLocation):
        # check if the file exists
        fileNode = {}
        if(self.fileExists(filename, fileLocation)):
            filePath = fileLocation.split(os.path.sep)[1:]
            directoryNode = self.tracePath(fileLocation)
            # we've reached the directory
            fileNode = self.selectFileFromDirectoryByName(filename, directoryNode)
            # update file's access node
            fileNode["file"].access_time = datetime.now()

        return fileNode
    
    def traceDirectory(self, path):
        currentNode = self.root

        if(len(path) >= 2):
            path = path[:-1]
        elif(len(path) == 1 and path[0] == ""):
            # it's the partition
            return currentNode
        
        for directory in path:
            # move the currentNode pointer to the current directory
            currentNode = [child for child in currentNode["children"] if child["name"] == directory]
            if(len(currentNode) == 0):
                return {}
            currentNode = currentNode[0]
        
        return currentNode
    
    def updateEachDirectorySizeInPath(self, fileLocation, newSize):
        path = fileLocation.split(os.path.sep)
        currentNode = self.root
        for directory in path[1:]:
            # update the size of the partition
            if(currentNode["type"] == "partition"):
                currentNode["partition"].usedSpace += newSize
                currentNode["partition"].freeSpace -= newSize
            # update the size of the directory
            elif(currentNode["type"] == "directory"): 
                currentNode["directory"].size += newSize
            # move the currentNode pointer to the current directory
            currentNode = [child for child in currentNode["children"] if child["name"] == directory][0]
        # update the directory's size
        currentNode["directory"].size += newSize
                        
    def createDirectoryIfDoesntExists(self, path, directory, node):
        dirChildren = [child["name"] for child in node["children"]]
        if directory not in dirChildren:
            if(directory != ""):
                print("- Creating new directory", directory)
                newDirectory = self.createNewDirectoryInstance(path, directory)
                node["children"].append({"name": directory, "type": "directory", "directory": newDirectory, "children": []})
            
        return node
        
    def createNewDirectoryInstance(self, path, directory):
        # generating the new directory location
        directoryLocation = ""
        for pathDirectory in path:
            if(pathDirectory != directory):
                directoryLocation += pathDirectory + "/"
            else:
                break
        # creating a new directory instance
        newDirectoryId = random.randint(0, 400)
        newDirectory = Directory(directory, newDirectoryId, directoryLocation, 0, "WR", datetime.now(), datetime.now(), datetime.now())

        return newDirectory
                                     
    def tracePath(self, fileLocation):
        path = fileLocation.split(os.path.sep)[1:]
        currentNode = self.root
        for directory in path:
            # if directory doesn't exist, create one
            currentNode = self.createDirectoryIfDoesntExists(path, directory, currentNode)
            # move the currentNode pointer to the current directory
            currentNode = [child for child in currentNode["children"] if child["name"] == directory][0]
                
        return currentNode
    
    def selectFileFromDirectoryByName(self, filename, directoryNode):
        file = {}
        for child in directoryNode["children"]:
            if(child["type"] == "file" and child["name"] == filename):
                file = child
                
        return file
    
    def deleteFileFromDirectory(self, filename, directoryNode):
        for child in directoryNode["children"]:
            if(child["type"] == "file" and child["name"] == filename):
                directoryNode["children"].remove(child)
                
        return directoryNode
    
    def calculateNumberOfBlockForEachFile(self, fileSize):
        blocks = 1
        while(True):
            fileSize -= self.blockSize
            if(fileSize < 0):
                return blocks
            blocks += 1
            
    def findInFileTree(self, results, search, currentNode):
        if(currentNode["type"] == "file"):
            if(currentNode["name"] == search):
                results.append(currentNode["file"].location)
                return
            return
            
        if(currentNode["name"] == search):
            results.append(currentNode[currentNode["type"]].location)
            
        for child in currentNode["children"]:
            self.findInFileTree(results, search, child)
            
    def generateFileTree(self, currentNode, level):
        if(currentNode["type"] == "file"):
            print('───'* level, currentNode["name"], "(", currentNode[currentNode["type"]].size, " bytes )")
            return
        
        if(currentNode["type"] == "partition"):
            print(currentNode["name"], "(", currentNode["partition"].usedSpace, "/" , currentNode["partition"].size , " bytes )")
        else:
            print('───' * level, currentNode["name"], "(", currentNode[currentNode["type"]].size, " bytes )")
        for child in currentNode["children"]:
            self.generateFileTree(child, level+1)

In [186]:
file_system = FileSystem()
file = File(
    name='example.txt',
    identifier=1,
    location='/home/user/documents',
    file_type='txt',
    size=13,
    blocks=0,
    protection='R',
    creation_time=datetime.now(),
    modification_time=datetime.now(),
    access_time=datetime.now(),
    content='Hello, world!'
)

In [187]:
# create, read and write
file_system.createFile(file)
file_system.readFile('example.txt', '/home/user/documents')
file_system.writeToFile('example.txt', '/home/user/documents', "Goodbye, world!")

- Creating new directory home
- Creating new directory user
- Creating new directory documents
- File example.txt was created at /home/user/documents
-----------------------
Content of file example.txt at directory /home/user/documents :
Hello, world!
-----------------------
- File example.txt at directory /home/user/documents - Read-only -
-----------------------


In [188]:
# change permission
file_system.changeFilePermissions('example.txt', '/home/user/documents', 'RW')
file_system.writeToFile('example.txt', '/home/user/documents', "Goodbye, world!")
file_system.readFile('example.txt', '/home/user/documents')

- File example.txt at /home/user/documents permission changed to RW
-----------------------
- Content of file example.txt at directory /home/user/documents has been modified
-----------------------
Content of file example.txt at directory /home/user/documents :
Goodbye, world!
-----------------------


In [189]:
# copy and move files
file_system.copyFile('example.txt', '/home/user/documents', '/home/user/backups')
file_system.copyFile('example.txt', '/home/user/backups', '/home/user/documents')
file_system.renameFile('example.txt', '/home/user/documents', 'newExample.txt')
file_system.readFile('example.txt', '/home/user/backups')
file_system.readFile('newExample.txt', '/home/user/documents')

- Creating new directory backups
- File example.txt at /home/user/documents was copied to /home/user/backups
-----------------------
- File example.txt at /home/user/backups was copied to /home/user/documents
-----------------------
- File example.txt at /home/user/documents was renamed to newExample.txt
-----------------------
Content of file example.txt at directory /home/user/backups :
Goodbye, world!
-----------------------
Content of file newExample.txt at directory /home/user/documents :
Goodbye, world!
-----------------------


In [190]:
# file properties 
file_system.getFileProperties('example.txt', '/home/user/documents')

File example.txt at /home/user/documents Properties:
- Identifier : 1
- Type : txt
- Size : 15
- Blocks : 4
- Protection : RW
- Creation time : 2023-12-27 18:04:52.272972
- Modification time : 2023-12-27 18:04:52.298174
- Access time : 2023-12-27 18:04:52.320238
- Content : Goodbye, world!
-----------------------


In [191]:
# create another file
noteFile = File(
    name='note.txt',
    identifier=2,
    location='/home/notes',
    file_type='txt',
    size=14,
    blocks=0,
    protection='W',
    creation_time=datetime.now(),
    modification_time=datetime.now(),
    access_time=datetime.now(),
    content='This is a note'
)

file_system.createFile(noteFile)
file_system.readFile('note.txt', '/home/notes')
file_system.changeFilePermissions('note.txt', '/home/notes', 'RW')
file_system.readFile('note.txt', '/home/notes')

- Creating new directory notes
- File note.txt was created at /home/notes
-----------------------
- File note.txt at directory /home/notes - Write-only -
-----------------------
- File note.txt at /home/notes permission changed to RW
-----------------------
Content of file note.txt at directory /home/notes :
This is a note
-----------------------


In [192]:
# create and delete directory
file_system.createDirectory("newFolder", "/home/user/")
file_system.deleteDirectory("/home/user/newFolder")

- Directory newFolder at /home/user/ was created
- Folder /home/user/newFolder was deleted
-----------------------


In [193]:
# change directory permissions
file_system.changeDirectoryPermissions('/home/notes', "W")

- Directory /home/notes permission changed to W
-----------------------


In [194]:
# Directory Properties
file_system.getDirectoryProperties("/home/notes")

Directory /home/notes Properties:
- Name : notes
- Identifier : 70
- Location : home/
- Size : 14
- Protection : WR
- Creation time : 2023-12-27 18:04:52.332971
- Modification time : 2023-12-27 18:04:52.332973
- Access time : 2023-12-27 18:04:52.332973


In [195]:
# search
file_system.searchInDirectories("user")
file_system.searchInDirectories("example.txt")
file_system.searchInDirectories("file.txt")

Found user in :
- home/
-----------------------
Found example.txt in :
- /home/user/documents
- /home/user/backups
-----------------------
No results for file.txt
-----------------------


In [196]:
# Directory content
file_system.getDirectoryContent("/home/")

Directory /home/ contains :
- user ( 45 bytes )
- notes ( 14 bytes )
-----------------------


In [197]:
# file tree
file_system.displayFileTree()

---------- File Tree ----------
C ( 59 / 5000  bytes )
─── home ( 59  bytes )
────── user ( 45  bytes )
───────── documents ( 30  bytes )
──────────── example.txt ( 15  bytes )
──────────── newExample.txt ( 15  bytes )
───────── backups ( 15  bytes )
──────────── example.txt ( 15  bytes )
────── notes ( 14  bytes )
───────── note.txt ( 14  bytes )
-------------------------------


In [198]:
def printLogo():
        print("""
        ______  _____  _____ ___  ___
        |  _  \|  _  ||  _  ||  \/  |
        | | | || | | || | | || .  . |
        | | | || | | || | | || |\/| |
        | |/ / \ \_/ /\ \_/ /| |  | |
        |___/   \___/  \___/ \_|  |_/""")

        print("""
            _,.---._       ,-,--.  
          ,-.' , -  `.   ,-.'-  _\ 
         /==/_,  ,  - \ /==/_ ,_.' 
        |==|   .=.     |\==\  \    
        |==|_ : ;=:  - | \==\ -\   
        |==| , '='     | _\==\ ,\  
         \==\ -    ,_ / /==/\/ _ | 
          '.='. -   .'  \==\ - , / 
            `--`--''     `--`---'  
        """)

In [199]:
class App:
    
    def __init__(self):
        self.fileSystem = FileSystem()
        self.menu = [{"choice": "Create new file", "func": self.createNewFile}, 
                {"choice": "Read file", "func": self.readFile}, 
                {"choice": "Write to file", "func": self.writeToFile},
                {"choice": "Change file's permission", "func": self.changeFilePermission}, 
                {"choice": "Rename file", "func": self.renameFile}, 
                {"choice": "File's properties", "func": self.getFileProperties},
                {"choice": "Copy file", "func": self.copyFile},
                {"choice": "Move file", "func": self.moveFile},
                {"choice": "Create new Folder", "func": self.createNewFolder} ,
                {"choice": "Delete existing folder", "func": self.deleteFolder}, 
                {"choice": "Change folder's permission", "func": self.changeFolderPermission},
                {"choice": "Folder's properties", "func": self.getFolderProperties},
                {"choice": "Search", "func": self.search},
                {"choice": "Shutdown"}]
        
    def start(self):
        self.fileSystem.displayFileTree()
        self.displayMainMenu()
        
    def displayMainMenu(self):
        print("\n")
        for i in range(len(self.menu)):
            print(i+1, "-", self.menu[i]["choice"])
        userInput = int(input("Your choice:"))
        
        print("-----------------------")
        print(self.menu[userInput-1]["choice"])
        
        if(self.menu[userInput-1]["choice"] != "Shutdown"):
            self.menu[userInput-1]["func"]()
            self.start()
            
    def createNewFile(self):
        fileDetails = ["Name", "Location", "File type", "Protection", "Content"]
        userInputs = self.getInputs(fileDetails)
        
        newFile = File(
            name = userInputs[0],
            identifier = random.randint(0, 400),
            location = userInputs[1],
            file_type = userInputs[2],
            protection = userInputs[3],
            content = userInputs[4],
            creation_time=datetime.now(),
            modification_time=datetime.now(),
            access_time=datetime.now(),
            size=0,
            blocks=0
        )
        self.fileSystem.createFile(newFile)
        
    def readFile(self):
        fileDetails = ["File", "Location"]
        userInputs = self.getInputs(fileDetails)
        self.fileSystem.readFile(userInputs[0], userInputs[1])
        
    def changeFilePermission(self):
        fileDetails = ["File", "Location", "New protection"]
        userInputs = self.getInputs(fileDetails)
        self.fileSystem.changeFilePermissions(userInputs[0], userInputs[1], userInputs[2])
        
    def writeToFile(self):
        fileDetails = ["File", "Location", "Content"]
        userInputs = self.getInputs(fileDetails)
        self.fileSystem.writeToFile(userInputs[0], userInputs[1], userInputs[2])

    def renameFile(self):
        fileDetails = ["File", "Location", "New name"]
        userInputs = self.getInputs(fileDetails)
        self.fileSystem.renameFile(userInputs[0], userInputs[1], userInputs[2])
        
    def getFileProperties(self):
        fileDetails = ["File", "Location"]
        userInputs = self.getInputs(fileDetails)
        self.fileSystem.getFileProperties(userInputs[0], userInputs[1])
        
    def copyFile(self):
        fileDetails = ["File", "Location", "Destination Location"]
        userInputs = self.getInputs(fileDetails)
        self.fileSystem.copyFile(userInputs[0], userInputs[1], userInputs[2])
        
    def moveFile(self):
        fileDetails = ["File", "Location", "Destination Location"]
        userInputs = self.getInputs(fileDetails)
        self.fileSystem.moveFile(userInputs[0], userInputs[1], userInputs[2])
        
    def createNewFolder(self):
        directoryDetails = ["Name", "Location"]
        userInputs = self.getInputs(directoryDetails)
        self.fileSystem.createDirectory(userInputs[0], userInputs[1])
        
    def deleteFolder(self):
        directoryDetails = ["Location"]
        userInputs = self.getInputs(directoryDetails)
        self.fileSystem.deleteDirectory(userInputs[0])
        
    def changeFolderPermission(self):
        directoryDetails = ["Location", "New protection"]
        userInputs = self.getInputs(directoryDetails)
        self.fileSystem.changeDirectoryPermissions(userInputs[0], userInputs[1])
        
    def getFolderProperties(self):
        directoryDetails = ["Location"]
        userInputs = self.getInputs(directoryDetails)
        self.fileSystem.getDirectoryProperties(userInputs[0])
        
    def search(self):
        searchDetails = ["Search"]
        userInputs = self.getInputs(searchDetails)
        self.fileSystem.searchInDirectories(userInputs[0])
    
    # helpers    
    def getInputs(self, inputArr):
        userInputs = []
        for i in range(len(inputArr)):
            userInput = input(inputArr[i] + ":")
            userInputs.append(userInput)
            
        return userInputs

In [200]:
#printLogo()
app = App()
app.start()


        ______  _____  _____ ___  ___
        |  _  \|  _  ||  _  ||  \/  |
        | | | || | | || | | || .  . |
        | | | || | | || | | || |\/| |
        | |/ / \ \_/ /\ \_/ /| |  | |
        |___/   \___/  \___/ \_|  |_/

            _,.---._       ,-,--.  
          ,-.' , -  `.   ,-.'-  _\ 
         /==/_,  ,  - \ /==/_ ,_.' 
        |==|   .=.     |\==\  \    
        |==|_ : ;=:  - | \==\ -\   
        |==| , '='     | _\==\ ,\  
         \==\ -    ,_ / /==/\/ _ | 
          '.='. -   .'  \==\ - , / 
            `--`--''     `--`---'  
        
---------- File Tree ----------
C ( 0 / 5000  bytes )
-------------------------------


1 - Create new file
2 - Read file
3 - Write to file
4 - Change file's permission
5 - Rename file
6 - File's properties
7 - Copy file
8 - Move file
9 - Create new Folder
10 - Delete existing folder
11 - Change folder's permission
12 - Folder's properties
13 - Search
14 - Shutdown


Your choice: 9


-----------------------
Create new Folder


Name: movies
Location: /


- Directory movies at / was created
---------- File Tree ----------
C ( 0 / 5000  bytes )
─── movies ( 0  bytes )
-------------------------------


1 - Create new file
2 - Read file
3 - Write to file
4 - Change file's permission
5 - Rename file
6 - File's properties
7 - Copy file
8 - Move file
9 - Create new Folder
10 - Delete existing folder
11 - Change folder's permission
12 - Folder's properties
13 - Search
14 - Shutdown


...

-----------------------
Delete existing folder


Location: /movies/AdamSandler


- Folder /movies/AdamSandler was deleted
-----------------------
---------- File Tree ----------
C ( 248 / 5000  bytes )
─── movies ( 248  bytes )
────── PaulThomasAnderson ( 248  bytes )
───────── PunchDrunkLove2002.mp4 ( 66  bytes )
───────── Magnolia.mp4 ( 182  bytes )
-------------------------------


1 - Create new file
2 - Read file
3 - Write to file
4 - Change file's permission
5 - Rename file
6 - File's properties
7 - Copy file
8 - Move file
9 - Create new Folder
10 - Delete existing folder
11 - Change folder's permission
12 - Folder's properties
13 - Search
14 - Shutdown


Your choice: 14


-----------------------
Shutdown
